In [ ]:
import pandas as pb

data = pb.read_excel('FAKSexport.xlsx')
#print(data)
print(data.shape)



In [ ]:
import pandas as pb

Smaldata = pb.read_excel('SmalFAKS.xlsx')
#print(data)
print(Smaldata.shape)


In [ ]:
import pandas as pd

# Konvertera alla kolumnnamn till strängar för att undvika TypeError
Smaldata.columns = Smaldata.columns.astype(str)
data.columns = data.columns.astype(str)

print("1.6")
# Sök efter kolumner som innehåller det specifika ordet
word_to_remove = 'checkbox'
columns_to_remove = [col for col in Smaldata.columns if word_to_remove in col]

print(Smaldata.shape)
print(data.shape)
print("2")
# Ta bort de hittade kolumnerna från DataFrame
data = data.drop(columns=columns_to_remove)

print(Smaldata.shape)
print(data.shape)
print("3")
# Spara den uppdaterade DataFrame till en ny Excel-fil eller samma fil
data.to_excel('updFAKSexp.xlsx', index=False)

In [ ]:
import pandas as pb
data = pb.read_excel('updFAKSexp.xlsx')
print(1)
#data.to_csv('updFAKSexpCSV.csv')
data_np = data.to_numpy()
print(data.shape)

In [ ]:
orgdata = data

In [ ]:
print(data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import numpy as np

model_name = "AI-Sweden-Models/gpt-sw3-126m"

#data = pb.read_excel('updFAKSexp.xlsx')
print("1.6")


data = data.astype(str)

#print(data.iloc[:, 0])
#print(data.iloc[:, 1:])

# Tokenisera texten med en tokeniserare
tokenizer = AutoTokenizer.from_pretrained(model_name)


input_tokens = [tokenizer.encode(input_text) for input_text in data.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [tokenizer.encode(output_text) for output_text in data.iloc[:, 0]]



par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

print(par_tokens[3])
print(len(par_tokens))

print("1.6")

# Dela upp datamängden enligt 80-10-10-split
train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)
print("1.6")

'''PatientJournalText_train, PatientJournalText_val_and_test = train_test_split(output_tokens, test_size=0.2, random_state=42)
PatientJournalText_val, PatientJournalText_test = train_test_split(PatientJournalText_val_and_test, test_size=0.5, random_state=42)
InputData_train, InputData_val_and_test = train_test_split(input_tokens, test_size=0.2, random_state=42)
InputData_val, InputData_test = train_test_split(InputData_val_and_test, test_size=0.5, random_state=42)         blir random och har då inte rätt par''' 


#print(test_texts)
#print(train_texts)

# Spara de delade datamängderna till separata filer
with open('train_data.txt', 'w') as f:
    for text in train_texts:
        f.write(' '.join(map(str, text)) + '\n')

with open('val_data.txt', 'w') as f:
    for text in val_texts:
        f.write(' '.join(map(str, text)) + '\n')


with open('test_data.txt', 'w') as f:
    for text in test_texts:
        f.write(' '.join(map(str, text)) + '\n')


In [ ]:
output_tokens_from_par_train = [item[0] for item in train_texts]
output_tokens_from_par_val = [item[0] for item in val_texts]
output_tokens_from_par_test = [item[0] for item in test_texts]
input_tokens_from_par_train = [item[1:] for item in train_texts]
input_tokens_from_par_val = [item[1:] for item in val_texts]
input_tokens_from_par_test = [item[1:] for item in test_texts]

PatientJournalText_train_small = output_tokens_from_par_train[:80]
PatientJournalText_val_small = output_tokens_from_par_val[:10]
PatientJournalText_test_small = output_tokens_from_par_test[:10]
InputData_train_small = input_tokens_from_par_train[:80]
InputData_val_small = input_tokens_from_par_val[:10]
InputData_test_small = input_tokens_from_par_test[:10]

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pb

input_tokens = [(input_text) for input_text in data.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [(output_text) for output_text in data.iloc[:, 0]]

par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train = [item[0] for item in train_texts]
output_tokens_from_par_val = [item[0] for item in val_texts]
output_tokens_from_par_test = [item[0] for item in test_texts]
input_tokens_from_par_train = [item[1:] for item in train_texts]
input_tokens_from_par_val = [item[1:] for item in val_texts]
input_tokens_from_par_test = [item[1:] for item in test_texts]

PatientJournalText_train_small = [str(item) for item in output_tokens_from_par_train[:80]]
PatientJournalText_val_small = output_tokens_from_par_val[:10]
PatientJournalText_test_small = output_tokens_from_par_test[:10]
InputData_train_small = [str(item) for item in input_tokens_from_par_train[:80]]
InputData_val_small = input_tokens_from_par_val[:10]
InputData_test_small = input_tokens_from_par_test[:10]

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, pipeline, AutoTokenizer, AutoModelForCausalLM
import os

# Set environment variables
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, PatientJournalText_small, InputData_small, tokenizer):
        self.PatientJournalText_small = PatientJournalText_small
        self.InputData_small = InputData_small
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.InputData_small)

    def __getitem__(self, idx):        
        PatientJournalText_small = self.PatientJournalText_small[idx]
        InputData_small = self.InputData_small[idx]

        # Encode input sequence and labels using tokenizer
        inputs = self.tokenizer.encode_plus(
            InputData_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=512,  # Set this according to your maximum sequence length
        )

        labels = self.tokenizer.encode_plus(
            PatientJournalText_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=512,  # Set this according to your maximum sequence length
        )

        return {
            "input_ids": inputs.input_ids.flatten(),
            "labels": labels.input_ids.flatten()
        }

# Assuming you have already initialized the tokenizer somewhere
#device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "AI-Sweden-Models/gpt-sw3-126m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
#model.to(device)

AllData_train_small = MyDataset(PatientJournalText_train_small, InputData_train_small, tokenizer)
AllData_val_small = MyDataset(PatientJournalText_val_small, InputData_val_small, tokenizer)
AllData_test_small = MyDataset(PatientJournalText_test_small, InputData_test_small, tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
import numpy as np
import evaluate


#model_name = "AI-Sweden-Models/gpt-sw3-126m"



training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    logging_steps=1,
    save_steps=1,
    logging_dir="./logs")
    #metric = evaluate.load("accuracy")

'''def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)'''

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset= AllData_train_small,
    #eval_dataset= PatientJournalText_val_small,
    #compute_metrics= compute_metrics,
)

trainer.train()


In [ ]:
print("----------------")
output_tokens_from_par = [item[0] for item in par_tokens]
print(output_tokens_from_par[0])
output_tokens_from_par = [item[1:] for item in par_tokens]
print(output_tokens_from_par[0])

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()